In [1]:
import sys
print(sys.version)

3.7.9 (default, Feb 19 2021, 20:27:40) 
[Clang 12.0.0 (clang-1200.0.32.29)]


In [6]:
sys.path.append("../")

In [7]:
import os, sqlite3, config, sys
import pandas as pd
import numpy as np
import backtrader as bt
# from report import Cerebro
from strategy_classes import CrossOver, OpeningRangeBreakout

In [8]:
stock_id = 9395
strategy = "crossover"
start_date = '2020-04-20'
end_date = '2020-07-20'
set_cash = 30000

In [9]:
conn = sqlite3.connect(config.DB_FILE)
cursor = conn.cursor()

In [ ]:
def saveplots(cerebro, numfigs=1, iplot=True, start=None, end=None,
             width=16, height=9, dpi=300, tight=True, use=None, file_path = '', **kwargs):

        from backtrader import plot
        if cerebro.p.oldsync:
            plotter = plot.Plot_OldSync(**kwargs)
        else:
            plotter = plot.Plot(**kwargs)

        figs = []
        for stratlist in cerebro.runstrats:
            for si, strat in enumerate(stratlist):
                rfig = plotter.plot(strat, figid=si * 100,
                                    numfigs=numfigs, iplot=iplot,
                                    start=start, end=end, use=use)
                figs.append(rfig)

        for fig in figs:
            for f in fig:
                f.savefig(file_path, bbox_inches='tight')
        return figs

In [ ]:
def backtest(stock_id, strategy, conn, start_date=None, end_date=None, \
             open_range=None, run_id=None,liquidate_time='15:00:00', set_cash=25000):
    
    print(f"== Testing {stock_id} ==")
    
    df = pd.read_sql("""
        SELECT datetime, open, high, low, close, volume
        FROM stock_price_minute
        WHERE stock_id = :stock_id
        AND strftime('%Y-%m-%d', datetime) >= :start_date
        AND strftime('%Y-%m-%d', datetime) <= :end_date
        ORDER BY datetime ASC
        LIMIT 10000
        """, conn, params={"stock_id":stock_id,"start_date":start_date, \
                           "end_date":end_date}, index_col='datetime', parse_dates=['datetime'])
    data = df.between_time('09:30:00', '16:00:00')
    
    # initialize Cerebro engine, extende with report method
    cerebro = Cerebro()
    cerebro.broker.setcash(set_cash)
    cerebro.addsizer(bt.sizers.PercentSizer, percents=95)
    
    # add data
    feed = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(feed)
    
    if strategy == 'opening_range_breakout':
        cerebro.addstrategy(strategy=OpeningRangeBreakout)
    else:
        # add Golden Cross strategy
        params = (('fast', 50),('slow', 200))
        cerebro.addstrategy(strategy=CrossOver, **dict(params))
        
    cerebro.run()
    
    saveplots(cerebro, file_path = 'backtest_output.png')
    
    cerebro.report(memo=f'{stock_id} | {run_id}',
               outputdir='/Users/kylespringfield/Dev/MoneyTree/backtest_reports')

In [ ]:
backtest(stock_id, strategy, conn, start_date=start_date, end_date=end_date, \
         run_id=run_id, set_cash=set_cash)

### sandbox

In [5]:
start_date = '2020-04-20'
end_date = '2020-04-22'

In [6]:
df = pd.read_sql("""
        SELECT datetime, open, high, low, close, volume
        FROM stock_price_minute
        WHERE stock_id = :stock_id
        AND strftime('%Y-%m-%d', datetime) >= :start_date
        AND strftime('%Y-%m-%d', datetime) <= :end_date
        ORDER BY datetime ASC
        LIMIT 500
    """, conn, params={"stock_id":9395,"start_date":start_date, \
                       "end_date":end_date}, index_col='datetime', parse_dates=['datetime'])
df.head()

,open,high,low,close,volume
datetime,,,,,
2020-04-20 04:00:00,70.680,70.680,70.500,70.500,5384.0
2020-04-20 04:01:00,70.680,70.680,70.500,70.500,5384.0
2020-04-20 04:02:00,70.680,70.680,70.500,70.500,5384.0
2020-04-20 04:03:00,70.680,70.680,70.500,70.500,5384.0
2020-04-20 04:04:00,70.375,70.425,70.375,70.425,1400.0


In [11]:
df = df.between_time('09:30:00', '16:00:00')

In [12]:
curve = df['open']
curve

datetime
2020-04-20 09:30:00    69.4875
2020-04-20 09:31:00    69.6559
2020-04-20 09:32:00    69.8200
2020-04-20 09:33:00    69.8950
2020-04-20 09:34:00    70.0225
                        ...   
2020-04-20 12:15:00    70.0194
2020-04-20 12:16:00    70.0400
2020-04-20 12:17:00    70.0525
2020-04-20 12:18:00    70.0875
2020-04-20 12:19:00    70.0200
Name: open, Length: 170, dtype: float64

In [14]:
df = pd.DataFrame(curve)
df = df.rename(columns = {0:"value"})

In [15]:
df.head()

,open
datetime,
2020-04-20 09:30:00,69.4875
2020-04-20 09:31:00,69.6559
2020-04-20 09:32:00,69.8200
2020-04-20 09:33:00,69.8950
2020-04-20 09:34:00,70.0225


In [ ]:
run_id = 1

In [ ]:
# PnL
start_cash = 20000
rpl = 1644.4
result_won_trades = 3587.16
result_lost_trades = -1942.77
profit_factor = 1.84
rpl_per_trade = 42.2
total_return = .0657
annual_return = .3814
max_money_drawdown = 907.8
max_pct_drawdown = .0349
# Trades
total_number_trades = 39
trades_closed = 35
pct_winning = .6153
pct_losing = .3846
avg_money_winning = 149.46
avg_money_losing = -129.52
best_winning_trade = 351.38
worst_losing_trade = -315.37
# KPIs
sharpe_ratio = 1.13
sqn_score = 1.58
sqn_human = "Poor"


In [ ]:
kpis = {# PnL
       'start_cash': start_cash,
       'rpl': rpl,
       'result_won_trades': result_won_trades,
       'result_lost_trades': result_lost_trades,
       'profit_factor': profit_factor,
       'rpl_per_trade': rpl_per_trade,
       'total_return': total_return,
       'annual_return': annual_return,
       'max_money_drawdown': max_money_drawdown,
       'max_pct_drawdown': max_pct_drawdown,
       # trades
       'total_number_trades': total_number_trades,
       'trades_closed': trades_closed,
       'pct_winning': pct_winning,
       'pct_losing': pct_losing,
       'avg_money_winning': avg_money_winning,
       'avg_money_losing':  avg_money_losing,
       'best_winning_trade': best_winning_trade,
       'worst_losing_trade': worst_losing_trade,
       #  performance
       'sharpe_ratio': sharpe_ratio,
       'sqn_score': sqn_score,
       'sqn_human': sqn_human
       }

In [ ]:
kpis['run_id'] = run_id
kpis

In [ ]:
rows =[]

In [ ]:
rows.append(kpis)

In [ ]:
kpis_df = pd.DataFrame.from_dict(rows)

In [ ]:
kpis_df

In [ ]:
kpis_df = kpis_df[['run_id', 'start_cash', 'rpl', 'result_won_trades', 'result_lost_trades', 'profit_factor', 'rpl_per_trade', 'total_return', 'annual_return', 'max_money_drawdown', 'max_pct_drawdown', 'total_number_trades', 'trades_closed', 'pct_winning', 'pct_losing', 'avg_money_winning', 'avg_money_losing', 'best_winning_trade', 'worst_losing_trade', 'sharpe_ratio', 'sqn_score', 'sqn_human']]

In [ ]:
kpis_df

In [ ]:
cursor.execute("""
    INSERT INTO backtest_reports 
    (run_id, start_cash, rpl, result_won_trades, result_lost_trades, profit_factor, rpl_per_trade,
     total_return, annual_return, max_money_drawdown, max_pct_drawdown, total_number_trades, trades_closed,
     pct_winning, pct_losing, avg_money_winning, avg_money_losing, best_winning_trade, worst_losing_trade,
     sharpe_ratio, sqn_score, sqn_human) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
     """, (int(kpis_df.run_id[0]), int(kpis_df.start_cash[0]), kpis_df.rpl[0], kpis_df.result_won_trades[0], kpis_df.result_lost_trades[0],
           kpis_df.profit_factor[0], kpis_df.rpl_per_trade[0], kpis_df.total_return[0], kpis_df.annual_return[0],
           kpis_df.max_money_drawdown[0], kpis_df.max_pct_drawdown[0], int(kpis_df.total_number_trades[0]),
           int(kpis_df.trades_closed[0]), kpis_df.pct_winning[0], kpis_df.pct_losing[0], kpis_df.avg_money_winning[0],
           kpis_df.avg_money_losing[0], kpis_df.best_winning_trade[0], kpis_df.worst_losing_trade[0],
           kpis_df.sharpe_ratio[0], kpis_df.sqn_score[0], kpis_df.sqn_human[0]))

In [ ]:
conn.commit()

In [ ]:
q = """
    SELECT *
    FROM backtest_reports
    """

df = pd.read_sql(q, conn)
df.head(20)

Filter curve data

In [62]:
df = pd.read_csv("data/curve_data.csv")

In [63]:
df.head()

,datetime,value
0,2020-04-14 00:00:00,100.0
1,2020-04-14 00:01:00,100.0
2,2020-04-14 00:02:00,100.0
3,2020-04-14 00:03:00,100.0
4,2020-04-14 00:04:00,100.0


In [64]:
df.set_index('datetime', inplace=True)

In [65]:
df.to

,value
datetime,
2020-04-14 00:00:00,100.0
2020-04-14 00:01:00,100.0
2020-04-14 00:02:00,100.0
2020-04-14 00:03:00,100.0
2020-04-14 00:04:00,100.0


In [71]:
df.index = pd.to_datetime(df.index)

In [72]:
len(df)

194004

In [73]:
market_end_df = df.between_time('00:04:00', '00:04:00')

In [74]:
len(market_end_df)

125

In [75]:
market_end_df.head()

,value
datetime,
2020-04-14 00:04:00,100.000000
2020-04-15 00:04:00,97.654125
2020-04-16 00:04:00,96.457240
2020-04-17 00:04:00,98.830204
2020-04-21 00:04:00,96.932749
